<a href="https://colab.research.google.com/github/neerajthandayan/Tensorflow-2.0/blob/main/CNN_%2B_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Loading and Pre-processing

In [1]:
# Impoting Libraries

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
#Fetching Data

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# Changing the Data Shape

X_train, X_test = X_train/255, X_test/255
X_train, X_test = np.expand_dims(X_train, -1), np.expand_dims(X_test,-1)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

## GridSearch 1: Neuron, Activation and Optimizer Tuning

In [4]:
# Building funtion for model

def CNN_model(act_funct='relu',neuron=64,optimizer='rmsprop',loss='categorical_crossentropy'):
  #Libraries
  import tensorflow as tf
  from tensorflow.keras.models import Model
  from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

  #Input
  i = Input(shape=(X_train[0].shape))

  #Convolution layers
  o = Conv2D(32, (3,3), activation=act_funct)(i)
  o = MaxPooling2D((2,2))(o)
  o = Conv2D(64, (3,3), activation=act_funct)(o)
  o = MaxPooling2D((2,2))(o)
  o = Conv2D(64, (3,3), activation=act_funct)(o)

  #Dense Layer
  o = Flatten()(o)
  o = Dense(neuron, activation=act_funct)(o)
  o = Dense(10, activation='softmax')(o)

  clf = Model(i,o)

  #Compiling Model
  clf.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
  
  return clf

In [ ]:
# Checcking Default Performance

#result = CNN_model().fit(X_train,y_train, validation_data=(X_test,y_test), epochs=10)
#print(np.array(result.history['accuracy']).mean(), np.array(result.history['val_accuracy']).mean())
#print(np.array(result.history['loss']).mean(), np.array(result.history['val_loss']).mean())

In [5]:
act_funct = ['relu',tf.keras.layers.LeakyReLU()]
neuron = [64, 128, 256, 512]
optimizer = ['rmsprop','adam','adagrad']
loss = ['categorical_crossentropy','binary_crossentropy']

param_grid = dict(act_funct=act_funct,neuron=neuron,optimizer=optimizer,loss=loss)
Classifier = KerasClassifier(build_fn=CNN_model,epochs=5, batch_size=64,validation_split=0.2)

grid = GridSearchCV(estimator=Classifier,param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/5
750/750 [==============================] - 7s 5ms/step - loss: 0.1179 - accuracy: 0.7916 - val_loss: 0.0154 - val_accuracy: 0.9794
Epoch 2/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0128 - accuracy: 0.9824 - val_loss: 0.0104 - val_accuracy: 0.9864
Epoch 3/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0075 - accuracy: 0.9902 - val_loss: 0.0086 - val_accuracy: 0.9872
Epoch 4/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0056 - accuracy: 0.9917 - val_loss: 0.0072 - val_accuracy: 0.9895
Epoch 5/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0039 - accuracy: 0.9946 - val_loss: 0.0074 - val_accuracy: 0.9902


In [6]:
#summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.989100 using {'act_funct': 'relu', 'loss': 'binary_crossentropy', 'neuron': 64, 'optimizer': 'rmsprop'}
0.988167 (0.001112) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 64, 'optimizer': 'rmsprop'}
0.986833 (0.001265) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 64, 'optimizer': 'adam'}
0.890933 (0.005430) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 64, 'optimizer': 'adagrad'}
0.987500 (0.002109) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 128, 'optimizer': 'rmsprop'}
0.987000 (0.001783) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 128, 'optimizer': 'adam'}
0.891533 (0.004321) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 128, 'optimizer': 'adagrad'}
0.988900 (0.001695) with: {'act_funct': 'relu', 'loss': 'categorical_crossentropy', 'neuron': 256, 'optimizer': 'rmsprop'}
0.986033 (0.002963) with: {'act_func

## GridSearch 2: Momentum and Learning Rate

In [7]:
# Building funtion for model (Lerning Rate & Momentum)

def CNN_model2(learning_rate=0.001,momentum=0.0):
  #Libraries
  import tensorflow as tf
  from tensorflow.keras.models import Model
  from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

  #Input
  i = Input(shape=(X_train[0].shape))

  #Convolution layers
  o = Conv2D(32, (3,3), activation='relu')(i)
  o = MaxPooling2D((2,2))(o)
  o = Conv2D(64, (3,3), activation='relu')(o)
  o = MaxPooling2D((2,2))(o)
  o = Conv2D(64, (3,3), activation='relu')(o)

  #Dense Layer
  o = Flatten()(o)
  o = Dense(512, activation='relu')(o)
  o = Dense(10, activation='softmax')(o)

  clf_2 = Model(i,o)

  #Compiling Model
  clf_2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate,momentum=momentum),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  return clf_2

In [9]:
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid_2 = dict(learning_rate=learning_rate, momentum=momentum)
Classifier_2 = KerasClassifier(build_fn=CNN_model2,epochs=5, batch_size=64,validation_split=0.2)

grid_2 = GridSearchCV(estimator=Classifier_2, param_grid=param_grid_2, n_jobs=-1)
grid_result_2 = grid_2.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/5
750/750 [==============================] - 4s 4ms/step - loss: 0.0960 - accuracy: 0.8286 - val_loss: 0.0121 - val_accuracy: 0.9827
Epoch 2/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0098 - accuracy: 0.9848 - val_loss: 0.0133 - val_accuracy: 0.9818
Epoch 3/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0061 - accuracy: 0.9915 - val_loss: 0.0099 - val_accuracy: 0.9867
Epoch 4/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0043 - accuracy: 0.9937 - val_loss: 0.0080 - val_accuracy: 0.9898
Epoch 5/5
750/750 [==============================] - 3s 4ms/step - loss: 0.0032 - accuracy: 0.9951 - val_loss: 0.0086 - val_accuracy: 0.9891


In [10]:
#summarize results
print("Best: %f using %s" % (grid_result_2.best_score_, grid_result_2.best_params_))
means = grid_result_2.cv_results_['mean_test_score']
stds = grid_result_2.cv_results_['std_test_score']
params = grid_result_2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.988850 using {'learning_rate': 0.001, 'momentum': 0.0}
0.988850 (0.001977) with: {'learning_rate': 0.001, 'momentum': 0.0}
0.987750 (0.002192) with: {'learning_rate': 0.001, 'momentum': 0.2}
0.988817 (0.001257) with: {'learning_rate': 0.001, 'momentum': 0.4}
0.987850 (0.001623) with: {'learning_rate': 0.001, 'momentum': 0.6}
0.986350 (0.001896) with: {'learning_rate': 0.001, 'momentum': 0.8}
0.984483 (0.002142) with: {'learning_rate': 0.001, 'momentum': 0.9}
0.976633 (0.004460) with: {'learning_rate': 0.01, 'momentum': 0.0}
0.973017 (0.009226) with: {'learning_rate': 0.01, 'momentum': 0.2}
0.796183 (0.341607) with: {'learning_rate': 0.01, 'momentum': 0.4}
0.448617 (0.417392) with: {'learning_rate': 0.01, 'momentum': 0.6}
0.753367 (0.319250) with: {'learning_rate': 0.01, 'momentum': 0.8}
0.110533 (0.004689) with: {'learning_rate': 0.01, 'momentum': 0.9}
0.104683 (0.007586) with: {'learning_rate': 0.1, 'momentum': 0.0}
0.102683 (0.004996) with: {'learning_rate': 0.1, 'momentum': 